In [1]:
import warnings
warnings.simplefilter(action='ignore')
import string
import numpy as np
import tqdm
from tqdm import tqdm_notebook as tqdm
import h5py
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras as keras
from keras import backend as K

from theano import ifelse

# import os
# os.environ['KERAS_BACKEND'] = 'theano'
# THEANO_FLAGS='device=cuda,floatX=float32'


from keras.models import Model, Sequential
from keras.layers import Input, InputLayer
from keras.layers.core import Dense, RepeatVector, Masking, Dropout
from keras.layers.merge import Concatenate
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers import Embedding
from keras.layers.pooling import GlobalMaxPooling1D

from layers.helpers import compute_mask

from layers import QuestionAttnGRU
from layers import SelfAttnGRU
from layers import PointerGRU
from layers import QuestionPooling
from layers import VariationalDropout
from layers import Slice
from layers import SharedWeight

Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def h5load(file_name, emb=False):
    path = 'Prepared data/'
    if not emb:
        path += 'test_'
    with h5py.File(path + file_name + '.h5', 'r') as hf:
        return hf[file_name][:]
context = h5load('context')
context_maxlen = context.shape[1]
answers = h5load('answers')
answer_maxlen = answers.shape[1]
questions = h5load('questions')
ques_maxlen = questions.shape[1]
begin = h5load('begin')
end = h5load('end')
embedding_matrix = h5load('embeddings', True)
word2ind = np.load('Prepared data/word2ind.npy').item()
ind2word = np.load('Prepared data/ind2word.npy').item()
n = context.shape[0]
emb_dim = 300
vocab_size = len(word2ind)

In [3]:
import model
from model import RNet, custom_objects

In [4]:
# model = RNet(hdim=45, dropout_rate=0.2, N=None, M=None, word2vec_dim=emb_dim)
# model.compile(optimizer='Adadelta',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'], options = run_opts)
# model.summary()
# loaded_model = load_model('Weights/model.h5', custom_objects=custom_objects())
# for i in range(len(model.layers)):
#     if i != 2 and i != 3:
#         weights = loaded_model.layers[i].get_weights()
#         model.layers[i].set_weights(weights)
#     else:
#         weights = model.layers[i].get_weights()
#         model.layers[i].set_weights(weights)

In [5]:
part = 1
batch = 50
X_train, y_train = [context[:part], questions[:part]], [begin[:part], end[:part]]
if part < 40000:
    X_valid, y_valid = [context[part:2* part], questions[part:2 * part]], [begin[part:2 * part], end[part: 2 * part]]
else:
    X_valid, y_valid = [context[part:], questions[part:]], [begin[part:], end[part:]]

In [6]:
# model = model.save('Weights/new_model.h5')
model = load_model('Weights/new_model.h5', custom_objects=custom_objects())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
epochs = 1
#model.load_weights('Weights/weights.hdf5')
checkpointer = ModelCheckpoint(filepath='Weights/new_weights.hdf5', verbose=1, save_best_only=True)
model_history = model.fit(X_train, y_train, validation_data=[X_valid, y_valid], verbose=1,
                         batch_size=batch, epochs=epochs, callbacks=[checkpointer])

Train on 1 samples, validate on 1 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[1702820,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Variable_67/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable_67, Const_8)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Variable_67/Assign', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/di

In [8]:
model.save('new_model.h5')

KeyboardInterrupt: 